<a href="https://colab.research.google.com/github/LeoProvorov/Data-science/blob/master/Market_Prediction_XGBoost_with_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Market Prediction: XGBoost with GPU


In [5]:
from google.colab import files

files.upload() # Loading archive from kaggle. Throwing the json file here

# File location for kaggle.com authentication and access rights
!mkdir -p ~/.kaggle
!mv -i kaggle.json ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

!kaggle competitions download -c jane-street-market-prediction

!ls

Saving kaggle.json to kaggle.json
  0% 0.00/441k [00:00<?, ?B/s]
100% 441k/441k [00:00<00:00, 66.9MB/s]
  0% 0.00/59.0 [00:00<?, ?B/s]
100% 59.0/59.0 [00:00<00:00, 60.4kB/s]
100% 2.61G/2.61G [00:45<00:00, 52.4MB/s]
100% 2.61G/2.61G [00:45<00:00, 61.8MB/s]
  0% 0.00/23.3k [00:00<?, ?B/s]
100% 23.3k/23.3k [00:00<00:00, 24.8MB/s]
 55% 9.00M/16.4M [00:01<00:01, 6.09MB/s]
100% 16.4M/16.4M [00:01<00:00, 11.3MB/s]
  0% 0.00/108k [00:00<?, ?B/s]
100% 108k/108k [00:00<00:00, 102MB/s]
competition.cpython-37m-x86_64-linux-gnu.so  features.csv  train.csv.zip
example_sample_submission.csv		     __init__.py
example_test.csv.zip			     sample_data


In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_columns', 500)


# Standard plotly imports
import plotly as py
import plotly.graph_objs as go
import plotly.tools as tls
from plotly.offline import iplot, init_notebook_mode
import cufflinks
import cufflinks as cf
import plotly.figure_factory as ff
import os


import warnings
warnings.filterwarnings("ignore")

In [7]:
import os

import numpy as np
import pandas as pd
from sklearn import preprocessing
import xgboost as xgb

In [8]:
print("XGBoost version:", xgb.__version__)

XGBoost version: 0.90


In [9]:
%%time
train = pd.read_csv('train.csv.zip')
features = pd.read_csv('features.csv')
example_test = pd.read_csv('example_test.csv.zip')
sample_prediction_df = pd.read_csv('example_sample_submission.csv')
print ("Data is loaded!")

Data is loaded!
CPU times: user 1min 45s, sys: 6.09 s, total: 1min 51s
Wall time: 1min 51s


In [10]:
print('train shape is {}'.format(train.shape))
print('features shape is {}'.format(features.shape))
print('example_test shape is {}'.format(example_test.shape))
print('sample_prediction_df shape is {}'.format(sample_prediction_df.shape))

train shape is (2390491, 138)
features shape is (130, 30)
example_test shape is (15219, 133)
sample_prediction_df shape is (15219, 2)


In [11]:
train.head()

,date,weight,resp_1,resp_2,resp_3,resp_4,resp,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,feature_79,feature_80,feature_81,feature_82,feature_83,feature_84,feature_85,feature_86,feature_87,feature_88,feature_89,feature_90,feature_91,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99,feature_100,feature_101,feature_102,feature_103,feature_104,feature_105,feature_106,feature_107,feature_108,feature_109,feature_110,feature_111,feature_112,feature_113,feature_114,feature_115,feature_116,feature_117,feature_118,feature_119,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,ts_id
0,0,0.000000,0.009916,0.014079,0.008773,0.001390,0.006270,1,-1.872746,-2.191242,-0.474163,-0.323046,0.014688,-0.002484,NaN,NaN,-0.989982,-1.055090,NaN,NaN,-2.667671,-2.001475,-1.703595,-2.196892,NaN,NaN,1.483295,1.307466,NaN,NaN,1.175200,0.967805,1.608410,1.319365,NaN,NaN,-0.515073,-0.448988,NaN,NaN,-2.429812,-2.206423,-3.593120,-2.868358,0.112697,0.053157,-0.539956,-0.692187,3.491282,-1.684889,1.337123,-0.328607,1.689207,-1.052243,-1.870885,-1.789342,-1.574173,-1.120820,-0.571920,-1.093033,0.703515,5.936281,NaN,3.315812,1.291338,2.468825,2.490069,-1.148239,-0.961935,-2.263944,-2.158765,-5.012022,-2.006825,-1.284090,-2.141697,-2.054935,-1.851203,-1.431184,-1.634481,NaN,-0.373934,NaN,0.559241,0.891368,0.271700,NaN,-1.521125,NaN,3.045337,3.260512,0.683558,NaN,-0.109194,NaN,0.488806,1.447504,-2.790902,NaN,1.158770,NaN,3.754522,7.137163,-1.863069,NaN,0.434466,NaN,-0.292035,0.317003,-2.605820,NaN,2.896986,NaN,1.485813,4.147254,-2.238831,NaN,-0.892724,NaN,-0.156332,0.622816,-3.921523,NaN,2.561593,NaN,3.457757,6.649580,-1.472686,NaN,NaN,1.168391,8.313583,1.782433,14.018213,2.653056,12.600292,2.301488,11.445807,0
1,0,16.673515,-0.002828,-0.003226,-0.007319,-0.011114,-0.009792,-1,-1.349537,-1.704709,0.068058,0.028432,0.193794,0.138212,NaN,NaN,-0.151877,-0.384952,NaN,NaN,1.225838,0.789076,1.110580,1.102281,NaN,NaN,-0.590600,-0.625682,NaN,NaN,-0.543425,-0.547486,-0.706600,-0.667806,NaN,NaN,0.910558,0.914465,NaN,NaN,2.137454,2.080459,2.819291,2.483965,-0.086755,-0.082687,0.368431,0.469196,5.711996,-2.215132,0.796703,-1.140081,0.716617,-0.059431,-0.198920,-0.326697,-0.381770,1.435607,3.401393,2.486748,-2.014598,-0.390588,NaN,-0.027262,-1.886927,-1.706450,-0.888236,-1.138294,-0.954461,-1.350633,-1.459546,-4.564815,-2.651966,-1.620014,-2.240625,-2.147273,-0.255224,3.202946,-0.535872,NaN,-0.050948,NaN,0.141089,0.058363,0.131190,NaN,-0.121239,NaN,0.677553,0.045842,-0.124616,NaN,-0.007004,NaN,-0.410491,-0.024323,-3.012654,NaN,1.157671,NaN,1.297679,1.281956,-2.427595,NaN,0.024913,NaN,-0.413607,-0.073672,-2.434546,NaN,0.949879,NaN,0.724655,1.622137,-2.209020,NaN,-1.332492,NaN,-0.586619,-1.040491,-3.946097,NaN,0.983440,NaN,1.357907,1.612348,-1.664544,NaN,NaN,-1.178850,1.777472,-0.915458,2.831612,-1.417010,2.297459,-1.304614,1.898684,1
2,0,0.000000,0.025134,0.027607,0.033406,0.034380,0.023970,-1,0.812780,-0.256156,0.806463,0.400221,-0.614188,-0.354800,NaN,NaN,5.448261,2.668029,NaN,NaN,3.836342,2.183258,3.902698,3.045431,NaN,NaN,-1.141082,-0.979962,NaN,NaN,-1.157585,-0.966803,

# Missing Values Count

In [12]:
missing_values_count = train.isnull().sum() # Get the number of empty values. columns
print (missing_values_count)
total_cells = np.product(train.shape) # prod(product) multiplies all values. Here we get the length of the array
total_missing = missing_values_count.sum() # all empty values
print("% of missing data = ", (total_missing/total_cells) * 100)

date              0
weight            0
resp_1            0
resp_2            0
resp_3            0
               ... 
feature_126    8853
feature_127    8853
feature_128    1921
feature_129    1921
ts_id             0
Length: 138, dtype: int64
% of missing data =  2.050000594444611


# Features Importance
#### In the [Jane Street: EDA of day 0 and feature importance](https://www.kaggle.com/carlmcbrideellis/jane-street-eda-of-day-0-and-feature-importance) notebook, [Carl McBride Ellis](https://www.kaggle.com/carlmcbrideellis) tried to get features importance with a very quick Permutation Importance using the Random Forest. It was only for day 0, I will find out if those features can improve our model for the rest of days. I choosed only Top 100 features in version 10 but the score goes down, so let's try Top 50 features.

In [13]:
features = ['feature_39', 'feature_43', 'feature_5', 'feature_37', 'feature_42', 'feature_3', 'feature_77', 
			'feature_44', 'feature_41', 'feature_40', 'feature_27', 'feature_55', 'feature_64', 'feature_6', 
			'feature_45', 'feature_4', 'feature_83', 'feature_120', 'feature_101', 'feature_63', 'feature_124', 
			'feature_11', 'feature_30', 'feature_107', 'feature_61', 'feature_29', 'feature_62', 'feature_60', 
			'feature_90', 'feature_102', 'feature_31', 'feature_113', 'feature_38', 'feature_95', 'feature_68', 
			'feature_89', 'feature_21', 'feature_59', 'feature_13', 'feature_121', 'feature_114', 'feature_119', 
			'feature_57', 'feature_67', 'feature_74', 'feature_20', 'feature_78', 'feature_35', 'feature_86', 
			'feature_56', 'feature_7', 'feature_53', 'feature_126', 'feature_98', 'feature_72', 'feature_128', 
			'feature_25', 'feature_33', 'feature_84', 'feature_36', 'feature_92', 'feature_65', 'feature_70', 
			'feature_23', 'feature_58', 'feature_125', 'feature_34', 'feature_122', 'feature_19', 'feature_66', 
			'feature_12', 'feature_17', 'feature_116', 'feature_32', 'feature_51', 'feature_28', 'feature_22', 
			'feature_117', 'feature_54', 'feature_80', 'feature_8', 'feature_104', 'feature_108', 'feature_47', 
			'feature_127', 'feature_9', 'feature_96', 'feature_52', 'feature_110', 'feature_123', 'feature_75', 
			'feature_24', 'feature_105', 'feature_69', 'feature_111', 'feature_81', 'feature_49', 'feature_15', 
			'feature_46', 'feature_18']


**Is the data balanced or not?**

In [14]:
train.weight

0           0.000000
1          16.673515
2           0.000000
3           0.000000
4           0.138531
             ...    
2390486     0.000000
2390487     0.000000
2390488     0.000000
2390489     0.283405
2390490     0.000000
Name: weight, Length: 2390491, dtype: float64

In [15]:
# I have taked this cell from https://www.kaggle.com/jazivxt/the-market-is-reactive
# And https://www.kaggle.com/drcapa/jane-street-market-prediction-starter-xgb

train = train[train['weight'] != 0]

train['action'] = ((train['weight'].values * train['resp'].values) > 0).astype('int')


X_train = train.loc[:, features[:25]]
y_train = train.loc[:, 'action']

In [16]:
X_train = X_train.fillna(-999)

In [18]:
x = train['action'].value_counts().index
y = train['action'].value_counts().values


trace2 = go.Bar(
     x=x ,
     y=y,
     marker=dict(
         color=y,
         colorscale = 'Viridis',
         reversescale = True
     ),
     name="Imbalance",    
 )
layout = dict(
     title="Data imbalance - action",
     #width = 900, height = 500,
     xaxis=go.layout.XAxis(
     automargin=True),
     yaxis=dict(
         showgrid=False,
         showline=False,
         showticklabels=True,
 #         domain=[0, 0.85],
     ), 
)
fig1 = go.Figure(data=[trace2], layout=layout)
iplot(fig1)

In [19]:
del train, x, y

Training
To activate GPU usage, simply use tree_method='gpu_hist' (took me an hour to figure out, I wish XGBoost documentation was clearer about that).

In [20]:
# The training part taked from here https://www.kaggle.com/xhlulu/ieee-fraud-xgboost-with-gpu-fit-in-40s

clf = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=5,
    learning_rate=0.05,
    #subsample=0.9,
    #colsample_bytree=0.7,
    missing=-999,
    random_state=2020,
    tree_method='gpu_hist'  # THE MAGICAL PARAMETER
)

In [21]:
%time clf.fit(X_train, y_train)

CPU times: user 48.3 s, sys: 37.4 s, total: 1min 25s
Wall time: 1min 26s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.05, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=-999, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=2020,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, tree_method='gpu_hist', verbosity=1)

In [71]:
from janestreet import make_env # Так и не удалось найти файл. Из за этого указывается неверный размер в предикторе в следующей ячейке

env = make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set

ModuleNotFoundError: ignored

In [22]:
for (test_df, sample_prediction_df) in iter_test:
    X_test = test_df.loc[:, features[:50]]
    X_test.fillna(-999, inplace = True)
    y_preds = clf.predict(X_test)
    sample_prediction_df.action = y_preds
    env.predict(sample_prediction_df)

NameError: ignored

In [62]:
!ls

competition.cpython-37m-x86_64-linux-gnu.so  features.csv  sample_data
example_sample_submission.csv		     __init__.py   train.csv.zip
example_test.csv.zip			     __pycache__
